In [ ]:
!pip install transformers datasets torch accelerate




  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install transformers datasets torch scikit-learn google-colab wget


In [ ]:

!pip install transformers datasets torch evaluate scikit-learn


import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


file_path = "/content/privacy_policy_large_dataset.csv"
dataset = pd.read_csv(file_path)

# Preprocess dataset
def preprocess_labels(df, threshold=3):
    """
    Convert category scores to binary labels (0 or 1) based on the threshold.
    Ensure labels are formatted as lists of floats.
    """
    df["labels"] = df.apply(
        lambda row: [
            float(row["clarity"] > threshold),
            float(row["transparency"] > threshold),
            float(row["accessibility"] > threshold),
            float(row["security"] > threshold),
            float(row["comprehensiveness"] > threshold),
        ],
        axis=1,
    )
    return df


dataset = preprocess_labels(dataset)


hf_dataset = Dataset.from_pandas(dataset[["text", "labels"]])

# Split into train and test sets
dataset_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split["train"]
test_dataset = dataset_split["test"]

# Tokenizer setup
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_and_format(batch):
    tokenized = tokenizer(batch["text"], padding=True, truncation=True)
    tokenized["labels"] = batch["labels"]
    return tokenized

train_dataset = train_dataset.map(tokenize_and_format, batched=True)
test_dataset = test_dataset.map(tokenize_and_format, batched=True)

# Define the model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=5,
    problem_type="multi_label_classification",
)

# Define metrics for evaluation
def compute_metrics(pred):
    logits, labels = pred
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


trainer.train()


metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)


trainer.save_model("./fine_tuned_model")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

sample_texts = [
    "your data is secured and not shared.",
]
tokenized_samples = tokenizer(sample_texts, padding=True, truncation=True, return_tensors="pt")
tokenized_samples = {key: value.to(device) for key, value in tokenized_samples.items()}


output = model(**tokenized_samples)
predictions = torch.sigmoid(output.logits).detach().cpu().numpy()
print("Predictions:", predictions)



Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-26-667de50c956d>:87: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.288652,0.860000,0.941906,1.000000,0.968743
2,No log,0.167796,0.870000,0.971654,1.000000,0.984842
3,No log,0.125129,0.950000,0.988150,1.000000,0.993909
4,No log,0.106475,1.000000,1.000000,1.000000,1.000000
5,No log,0.100672,1.000000,1.000000,1.000000,1.000000


Evaluation Metrics: {'eval_loss': 0.10067180544137955, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.3806, 'eval_samples_per_second': 525.498, 'eval_steps_per_second': 34.157, 'epoch': 5.0}
Predictions: [[0.94735694 0.9572821  0.93929577 0.94128674 0.9507809 ]]


In [ ]:
!pip install transformers huggingface_hub



In [ ]:
from huggingface_hub import login


login()


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_path = "./fine_tuned_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


model.push_to_hub("ayaalhaj/privacy-policy-analyzer_multilabel")
tokenizer.push_to_hub("ayaalhaj/privacy-policy-analyzer_multilabel")


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ayaalhaj/privacy-policy-analyzer_multilabel/commit/05c8903cd1a1474b01f3ef3191e62cbd1f0ddb06', commit_message='Upload tokenizer', commit_description='', oid='05c8903cd1a1474b01f3ef3191e62cbd1f0ddb06', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ayaalhaj/privacy-policy-analyzer_multilabel', endpoint='https://huggingface.co', repo_type='model', repo_id='ayaalhaj/privacy-policy-analyzer_multilabel'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import snapshot_download


repo_id = "ayaalhaj/privacy-policy-analyzer_multilabel"
snapshot_download(repo_id=repo_id, local_dir="./transformers-js-model")


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'/content/transformers-js-model'